In [2]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, optimizers, callbacks
from tensorflow.keras.models import Sequential, Model
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os


In [14]:
folder_path = "/home/thomas/code/ThomasLechanoine/Breast_Cancer_Project/Data/Deep_learning"

train_path = os.path.join(folder_path, "train")
valid_path = os.path.join(folder_path, "valid")
test_path = os.path.join(folder_path, "test")

In [15]:
train_dataset = image_dataset_from_directory(
    train_path,
    labels="inferred",
    label_mode="binary",
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed=123
)

validation_dataset = image_dataset_from_directory(
    valid_path,
    labels="inferred",
    label_mode="binary",
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed=123
)

test_dataset = image_dataset_from_directory(
    test_path,
    labels="inferred",
    label_mode="binary",
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed=123
)

Found 2372 files belonging to 2 classes.


W0000 00:00:1741692895.461638   73311 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 675 files belonging to 2 classes.
Found 336 files belonging to 2 classes.


In [19]:
model = Sequential()

model.add(layers.Input((150, 150, 3)))
model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation="relu", padding="same"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu", padding="same"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same"))
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu", padding="same"))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation="relu"))

model.add(layers.Dropout(0.5))

model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 150, 150, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 75, 75, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 75, 75, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 37, 37, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 37, 37, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 43808)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     2,803,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,908,929 (11.10 MB)

 Trainable params: 2,908,929 (11.10 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5868 - loss: 2.8786  

75/75 ━━━━━━━━━━━━━━━━━━━━ 99s 1s/step - accuracy: 0.5873 - loss: 2.8584 - val_accuracy: 0.6667 - val_loss: 0.6385
Epoch 2/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step - accuracy: 0.6689 - loss: 0.6341 - val_accuracy: 0.6593 - val_loss: 0.6552
Epoch 3/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 130s 2s/step - accuracy: 0.6657 - loss: 0.6332 - val_accuracy: 0.6578 - val_loss: 0.6581
Epoch 4/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 111s 1s/step - accuracy: 0.6690 - loss: 0.6310 - val_accuracy: 0.6607 - val_loss: 0.6342
Epoch 5/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.6994 - loss: 0.5861 - val_accuracy: 0.6607 - val_loss: 0.6562
Epoch 6/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.6833 - loss: 0.5644 - val_accuracy: 0.6548 - val_loss: 0.6690
Epoch 7/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 107s 1s/step - accuracy: 0.7074 - loss: 0.5346 - val_accuracy: 0.6652 - val_loss: 0.7018
Epoch 8/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 126s 2s/step - accuracy: 0.7101 - loss: 0.5233 - val_accuracy: 0.6533 - val_loss: 0.6

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print("Test accuracy:", test_acc)

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()